# Import software libraries

In [29]:
# Import required libraries.
import sys           # Read system parameters.
import pandas as pd  # Manipulate and analyze data.
import sqlite3       # Manage SQL databases.

# Summarize software libraries used.
print('Libraries used in this project:')
print('- Python {}'.format(sys.version))
print('- pandas {}'.format(pd.__version__))
print('- sqlite3 {}'.format(sqlite3.sqlite_version))

Libraries used in this project:
- Python 3.7.6 | packaged by conda-forge | (default, Mar 23 2020, 23:03:20) 
[GCC 7.3.0]
- pandas 1.1.3
- sqlite3 3.30.1


# Examine the database

In [30]:
# Connect to SQLite database.

conn = sqlite3.connect('data/prod_sample.db')
conn

In [31]:
# List all the tables in the database.
cursor = conn.cursor()

cursor.execute("select name from sqlite_master where type='table';")

tables=cursor.fetchall()

print("Tables in prod_sample database:")
for table in tables:
    print(table[0])



Tables in prod_sample database:
stock_description
online_retail_history


# Read data from the `online_retail_history` table

In [32]:
# Write the query to be executed that selects everything from the online_retail_history table.
query = 'select * from online_retail_history'


# Use the read_sql function in pandas to read a query into a DataFrame.

ret_hist = pd.read_sql(query, conn)

# Preview the first five rows of the data.

ret_hist.head()

,Invoice,StockCode,Quantity,InvoiceDate,Price,CustomerID,Country,TotalAmount
0,536365,85123A,6,2010-12-01 08:26:00,2.55,u1785,United Kingdom,15.30
1,536367,84879,32,2010-12-01 08:34:00,1.69,u13047,United Kingdom,54.08
2,536373,85123A,6,2010-12-01 09:02:00,2.55,u1785,United Kingdom,15.30
3,536375,85123A,6,2010-12-01 09:32:00,2.55,u1785,United Kingdom,15.30
4,536378,20725,10,2010-12-01 09:37:00,1.65,u14688,United Kingdom,16.50


In [33]:
# Get the shape of the data.

ret_hist.shape

(15321, 8)

# Read data from the `stock_description` table

In [34]:
# Write the query to be executed that selects everything from the online_retail_history table.
query = 'select * from stock_description'



# Use the read_sql function in pandas to read a query into a DataFrame.

stock = pd.read_sql(query, conn)

# Preview the first five rows of the data.
stock.head()


,StockCode,Description
0,10002,INFLATABLE POLITICAL GLOBE
1,10080,GROOVY CACTUS INFLATABLE
2,10120,DOGGY RUBBER
3,10123C,HEARTS WRAPPING TAPE
4,10124A,SPOTS ON RED BOOKCOVER TAPE


In [35]:
# Get the shape of the data.

stock.shape

(3952, 2)

# Aggregate the `online_retail_history` and `stock_description` datasets

In [36]:
# Write a query to aggregate the two datasets so that you have the stock descriptions as well as the stock code.
query = 'select online_retail_history.*, stock_description.Description from online_retail_history left join stock_description on online_retail_history.StockCode = stock_description.StockCode'


# Use the read_sql function in pandas to read a query into a DataFrame.

ret_w_stock = pd.read_sql(query, conn)

# Preview the first five rows of the data.

ret_w_stock.head()

,Invoice,StockCode,Quantity,InvoiceDate,Price,CustomerID,Country,TotalAmount,Description
0,536365,85123A,6,2010-12-01 08:26:00,2.55,u1785,United Kingdom,15.30,CREAM HANGING HEART T-LIGHT HOLDER
1,536367,84879,32,2010-12-01 08:34:00,1.69,u13047,United Kingdom,54.08,ASSORTED COLOUR BIRD ORNAMENT
2,536373,85123A,6,2010-12-01 09:02:00,2.55,u1785,United Kingdom,15.30,CREAM HANGING HEART T-LIGHT HOLDER
3,536375,85123A,6,2010-12-01 09:32:00,2.55,u1785,United Kingdom,15.30,CREAM HANGING HEART T-LIGHT HOLDER
4,536378,20725,10,2010-12-01 09:37:00,1.65,u14688,United Kingdom,16.50,LUNCH BAG RED RETROSPOT


In [37]:
# Get the shape of the data.

ret_w_stock.shape

(17032, 9)

# Identify and fix corrupt or unusable data

In [38]:
# Check the value counts of the "Description" field.
ret_w_stock.Description.nunique()



11

In [39]:
#CHECKING HOW MANY ROWS CONTAIN '?' AS A VALUE IN DESCRIPTION FIELD
ret_w_stock[ret_w_stock.Description =='?']

,Invoice,StockCode,Quantity,InvoiceDate,Price,CustomerID,Country,TotalAmount,Description
36,536477,22423,16,2010-12-01 12:27:00,10.95,u1621,United Kingdom,175.20,?
40,536502,22423,2,2010-12-01 12:36:00,12.75,u16552,United Kingdom,25.50,?
45,536525,22423,2,2010-12-01 12:54:00,12.75,u14078,United Kingdom,25.50,?
48,536528,22423,1,2010-12-01 13:17:00,12.75,u15525,United Kingdom,12.75,?
50,536530,22423,1,2010-12-01 13:21:00,12.75,u17905,United Kingdom,12.75,?
...,...,...,...,...,...,...,...,...,...
16993,581449,22423,1,2011-12-08 17:37:00,12.75,u12748,United Kingdom,12.75,?
16995,581449,22423,1,2011-12-08 17:37:00,12.75,u12748,United Kingdom,12.75,?
17004,581472,22423,2,2011-12-08 19:55:00,12.75,u15796,United Kingdom,25.50,?
17016,581495,22423,10,2011-12-09 10:20:00,12.75,u14051,United Kingdom,127.50,?


In [40]:
# Remove rows where "Description" is just a question mark (?).
ret_w_stock_cleaned = ret_w_stock[ret_w_stock.Description !='?']


# Preview the first five rows of the data.
ret_w_stock_cleaned.head()



,Invoice,StockCode,Quantity,InvoiceDate,Price,CustomerID,Country,TotalAmount,Description
0,536365,85123A,6,2010-12-01 08:26:00,2.55,u1785,United Kingdom,15.30,CREAM HANGING HEART T-LIGHT HOLDER
1,536367,84879,32,2010-12-01 08:34:00,1.69,u13047,United Kingdom,54.08,ASSORTED COLOUR BIRD ORNAMENT
2,536373,85123A,6,2010-12-01 09:02:00,2.55,u1785,United Kingdom,15.30,CREAM HANGING HEART T-LIGHT HOLDER
3,536375,85123A,6,2010-12-01 09:32:00,2.55,u1785,United Kingdom,15.30,CREAM HANGING HEART T-LIGHT HOLDER
4,536378,20725,10,2010-12-01 09:37:00,1.65,u14688,United Kingdom,16.50,LUNCH BAG RED RETROSPOT


In [41]:
#Preview the shape of the data (to observe that the number of rows of this present table are now equal to that of the online_retail_history table)

ret_w_stock_cleaned.shape

(15321, 9)

# Identify and remove duplicates

In [42]:
# Identify all duplicated data.
duplicated_data = ret_w_stock_cleaned[ret_w_stock_cleaned.duplicated(keep=False)]




In [43]:
# Print the duplicated data.
print('Number of duplicated rows:', duplicated_data.shape[0])


Number of duplicated rows: 223


In [44]:
#Preview the first five rows of the duplicated data
duplicated_data.head()

,Invoice,StockCode,Quantity,InvoiceDate,Price,CustomerID,Country,TotalAmount,Description
176,536863,20727,1,2010-12-03 11:19:00,1.65,u17967,United Kingdom,1.65,LUNCH BAG BLACK SKULL.
178,536863,20727,1,2010-12-03 11:19:00,1.65,u17967,United Kingdom,1.65,LUNCH BAG BLACK SKULL.
496,537781,84879,8,2010-12-08 12:46:00,1.69,u17341,United Kingdom,13.52,ASSORTED COLOUR BIRD ORNAMENT
497,537781,84879,8,2010-12-08 12:46:00,1.69,u17341,United Kingdom,13.52,ASSORTED COLOUR BIRD ORNAMENT
569,537955,20725,1,2010-12-09 11:28:00,1.65,u16782,United Kingdom,1.65,LUNCH BAG RED RETROSPOT


In [45]:
# Remove the duplicated data.
ret_w_stock_cleaned_final = ret_w_stock_cleaned[~ret_w_stock_cleaned.duplicated()]



# Preview the first five rows of the data.

ret_w_stock_cleaned_final.head()


,Invoice,StockCode,Quantity,InvoiceDate,Price,CustomerID,Country,TotalAmount,Description
0,536365,85123A,6,2010-12-01 08:26:00,2.55,u1785,United Kingdom,15.30,CREAM HANGING HEART T-LIGHT HOLDER
1,536367,84879,32,2010-12-01 08:34:00,1.69,u13047,United Kingdom,54.08,ASSORTED COLOUR BIRD ORNAMENT
2,536373,85123A,6,2010-12-01 09:02:00,2.55,u1785,United Kingdom,15.30,CREAM HANGING HEART T-LIGHT HOLDER
3,536375,85123A,6,2010-12-01 09:32:00,2.55,u1785,United Kingdom,15.30,CREAM HANGING HEART T-LIGHT HOLDER
4,536378,20725,10,2010-12-01 09:37:00,1.65,u14688,United Kingdom,16.50,LUNCH BAG RED RETROSPOT


# Correct date formats

In [46]:
# Get the data types for every column in the DataFrame.
ret_w_stock_cleaned_final.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 15206 entries, 0 to 17031
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Invoice      15206 non-null  object 
 1   StockCode    15206 non-null  object 
 2   Quantity     15206 non-null  int64  
 3   InvoiceDate  15206 non-null  object 
 4   Price        15194 non-null  float64
 5   CustomerID   12435 non-null  object 
 6   Country      15206 non-null  object 
 7   TotalAmount  15194 non-null  float64
 8   Description  15206 non-null  object 
dtypes: float64(2), int64(1), object(6)
memory usage: 1.2+ MB


In [47]:
# Convert "InvoiceDate" to a "%Y-%m-%d" datetime format.
ret_w_stock_cleaned_2 = ret_w_stock_cleaned_final.copy()     #Work with a new data

ret_w_stock_cleaned_2['InvoiceDate'] = pd.to_datetime(ret_w_stock_cleaned_2['InvoiceDate'], format = '%Y-%m-%d')



In [48]:
# Get the data types for every column in the converted DataFrame.
ret_w_stock_cleaned_2.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 15206 entries, 0 to 17031
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   Invoice      15206 non-null  object        
 1   StockCode    15206 non-null  object        
 2   Quantity     15206 non-null  int64         
 3   InvoiceDate  15206 non-null  datetime64[ns]
 4   Price        15194 non-null  float64       
 5   CustomerID   12435 non-null  object        
 6   Country      15206 non-null  object        
 7   TotalAmount  15194 non-null  float64       
 8   Description  15206 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(5)
memory usage: 1.2+ MB


# Examine the table before finishing

In [49]:
# Preview the first five rows of the data.
ret_w_stock_cleaned_2.head()


,Invoice,StockCode,Quantity,InvoiceDate,Price,CustomerID,Country,TotalAmount,Description
0,536365,85123A,6,2010-12-01 08:26:00,2.55,u1785,United Kingdom,15.30,CREAM HANGING HEART T-LIGHT HOLDER
1,536367,84879,32,2010-12-01 08:34:00,1.69,u13047,United Kingdom,54.08,ASSORTED COLOUR BIRD ORNAMENT
2,536373,85123A,6,2010-12-01 09:02:00,2.55,u1785,United Kingdom,15.30,CREAM HANGING HEART T-LIGHT HOLDER
3,536375,85123A,6,2010-12-01 09:32:00,2.55,u1785,United Kingdom,15.30,CREAM HANGING HEART T-LIGHT HOLDER
4,536378,20725,10,2010-12-01 09:37:00,1.65,u14688,United Kingdom,16.50,LUNCH BAG RED RETROSPOT


# Load the dataset into a pickle file

In [50]:
# Save the dataset as a pickle file named online_history_cleaned.pickle.
ret_w_stock_cleaned_2.to_pickle('online_history_cleaned.pickle')

pd.read_pickle('online_history_cleaned.pickle').head()


,Invoice,StockCode,Quantity,InvoiceDate,Price,CustomerID,Country,TotalAmount,Description
0,536365,85123A,6,2010-12-01 08:26:00,2.55,u1785,United Kingdom,15.30,CREAM HANGING HEART T-LIGHT HOLDER
1,536367,84879,32,2010-12-01 08:34:00,1.69,u13047,United Kingdom,54.08,ASSORTED COLOUR BIRD ORNAMENT
2,536373,85123A,6,2010-12-01 09:02:00,2.55,u1785,United Kingdom,15.30,CREAM HANGING HEART T-LIGHT HOLDER
3,536375,85123A,6,2010-12-01 09:32:00,2.55,u1785,United Kingdom,15.30,CREAM HANGING HEART T-LIGHT HOLDER
4,536378,20725,10,2010-12-01 09:37:00,1.65,u14688,United Kingdom,16.50,LUNCH BAG RED RETROSPOT


In [51]:
#To get the datatypes for every column in the pickle file

pd.read_pickle('online_history_cleaned.pickle').info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15206 entries, 0 to 17031
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   Invoice      15206 non-null  object        
 1   StockCode    15206 non-null  object        
 2   Quantity     15206 non-null  int64         
 3   InvoiceDate  15206 non-null  datetime64[ns]
 4   Price        15194 non-null  float64       
 5   CustomerID   12435 non-null  object        
 6   Country      15206 non-null  object        
 7   TotalAmount  15194 non-null  float64       
 8   Description  15206 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(5)
memory usage: 1.2+ MB


In [52]:
# Close any connections to the database.
conn.close()
